In [1]:
import requests
import pandas as pd
import IPython
import random
import re
import mysql.connector

In [19]:
rows = 500
r = requests.get("https://archive.org/advancedsearch.php?q=collection%3A%22GratefulDead%22&rows={}&output=json".format(rows))
data = r.json()['response']['docs']
raw_concert_df = pd.DataFrame(data)

In [20]:
raw_concert_df.head()

,backup_location,collection,coverage,creator,date,description,downloads,format,identifier,indexflag,...,title,type,week,year,avg_rating,btih,num_reviews,publisher,reviewdate,stripped_tags
0,ia906708_15,"[GratefulDead, etree]","Denver, CO",Grateful Dead,1990-12-12T00:00:00Z,Set 1 Touch Of Grey Greatest Story Ever Told C...,334,"[Archive BitTorrent, Checksums, Columbia Peaks...",gd1990-12-12.142549.dts.tobin.flac1644,"[index, nonoindex]",...,Grateful Dead Live at McNichols Sports Arena o...,sound,0,1990,NaN,NaN,NaN,NaN,NaN,NaN
1,ia903600_5,"[GratefulDead, etree, fav-camrrt, fav-murgiema...","Philadelphia, PA",Grateful Dead,1974-08-04T00:00:00Z,"Bertha, Mexicali Blues, Scarlet Begonias, Blac...",67730,"[Archive BitTorrent, Checksums, Columbia Peaks...",gd74-08-04.aud-moore.weiner.20369.sbeok.shnf,"[index, nonoindex]",...,Grateful Dead Live at Philadelphia Civic Arena...,sound,40,1974,4.43,50877e179eaa75291c51d060a66b95ca4cdebe16,8.0,Jerry Moore,2017-10-26T20:06:34Z,NaN
2,ia903600_5,"[GratefulDead, etree, stream_only, fav-danfran...","St. Louis, MO",Grateful Dead,1972-10-17T00:00:00Z,"Promised Land, Bird Song, El Paso, Sugaree, Me...",41382,"[Archive BitTorrent, Checksums, Columbia Peaks...",gd72-10-17.sbd.sacks.2219.sbeok.shnf,"[index, nonoindex]",...,Grateful Dead Live at Fox Theatre on 1972-10-17,sound,49,1972,4.23,NaN,13.0,NaN,2022-01-30T04:42:12Z,NaN
3,ia903605_4,"[GratefulDead, etree, stream_only, fav-dean106...","San Francisco, CA",Grateful Dead,1966-05-19T00:00:00Z,"Set 1 Beat It On Down The Line, Standing On Th...",17099,"[Archive BitTorrent, Checksums, Flac, Flac Fin...",gd1966-05-19.sbd.miller.106683.flac16,"[index, nonoindex]",...,Grateful Dead Live at Avalon Ballroom on 1966-...,sound,11,1966,4.38,NaN,8.0,NaN,2015-05-19T14:00:14Z,NaN
4,ia903600_8,"[GratefulDead, etree, stream_only, fav-danesth...","Los Angeles, CA",Grateful Dead,1966-03-12T00:00:00Z,Viola Lee Blues One Kind Favor I Know You Ride...,26865,"[Archive BitTorrent, Checksums, Columbia Peaks...",gd1966-03-12.sbd.bershaw.9516.shnf,"[index, nonoindex]",...,Grateful Dead Live at Danish Center on 1966-03-12,sound,23,1966,3.83,NaN,6.0,NaN,2017-08-05T12:01:06Z,NaN


In [21]:
clean_concert_df = raw_concert_df[['title', 'year', 'coverage', 'identifier','date']]

In [12]:
#clean_concert_df = clean_concert_df.rename(columns={"title":"concert_title", "identifier":"concert_archive_id"})

,concert_title,year,coverage,concert_archive_id,date
0,Grateful Dead Live at McNichols Sports Arena o...,1990,"Denver, CO",gd1990-12-12.142549.dts.tobin.flac1644,1990-12-12T00:00:00Z
1,Grateful Dead Live at Philadelphia Civic Arena...,1974,"Philadelphia, PA",gd74-08-04.aud-moore.weiner.20369.sbeok.shnf,1974-08-04T00:00:00Z
2,Grateful Dead Live at Fox Theatre on 1972-10-17,1972,"St. Louis, MO",gd72-10-17.sbd.sacks.2219.sbeok.shnf,1972-10-17T00:00:00Z
3,"Grateful Dead Live at Dillon Gym, Princeton Un...",1971,"Princeton, NJ",gd1971-04-17.dts.mtx.tobin.107385.flac16,1971-04-17T00:00:00Z
4,Grateful Dead Live at Hollywood Palladium on 1...,1971,"Hollywood, CA",gd71-08-06.aud.bertrando.yerys.129.sbeok.shnf,1971-08-06T00:00:00Z
...,...,...,...,...,...
495,Grateful Dead Live at Winterland Arena on 1979...,1979,"San Francisco, CA",gd1979-12-29.137650.sen421.miller.sirmick.sbeo...,1979-12-29T00:00:00Z
496,Grateful Dead Live at Oakland Auditorium Arena...,1979,"Oakland, CA",gd1979-12-28.154878.akg.se5.wiseman.miller.clu...,1979-12-28T00:00:00Z
497,Grateful Dead Live at Oakland Auditorium Arena...,1979,"Oakland, CA",gd1979-12-31.144796.fob.sony.ecm270.severson.m...,1979-12-31T00:00:00Z
498,Grateful Dead Live at Capitol Theatre on 1980-...,1980,"Passaic, NJ",gd1980-03-30.sbd.miller.92666.sbeok.flac16,1980-03-30T00:00:00Z


In [22]:
clean_concert_df

,title,year,coverage,identifier,date
0,Grateful Dead Live at McNichols Sports Arena o...,1990,"Denver, CO",gd1990-12-12.142549.dts.tobin.flac1644,1990-12-12T00:00:00Z
1,Grateful Dead Live at Philadelphia Civic Arena...,1974,"Philadelphia, PA",gd74-08-04.aud-moore.weiner.20369.sbeok.shnf,1974-08-04T00:00:00Z
2,Grateful Dead Live at Fox Theatre on 1972-10-17,1972,"St. Louis, MO",gd72-10-17.sbd.sacks.2219.sbeok.shnf,1972-10-17T00:00:00Z
3,Grateful Dead Live at Avalon Ballroom on 1966-...,1966,"San Francisco, CA",gd1966-05-19.sbd.miller.106683.flac16,1966-05-19T00:00:00Z
4,Grateful Dead Live at Danish Center on 1966-03-12,1966,"Los Angeles, CA",gd1966-03-12.sbd.bershaw.9516.shnf,1966-03-12T00:00:00Z
...,...,...,...,...,...
495,Grateful Dead Live at Copps Coliseum on 1990-0...,1990,"Hamilton, ON",gd1990-03-21.156422.UltraMatrix.sbd.cm.miller....,1990-03-21T00:00:00Z
496,Grateful Dead Live at Copps Coliseum on 1990-0...,1990,"Hamilton, ON",gd1990-03-22.156423.UltraMatrix.sbd.cm.miller....,1990-03-22T00:00:00Z
497,Grateful Dead Live at Copps Coliseum on 1990-0...,1990,"Hamilton, ON",gd1990-03-21.141930.OTS.Schoeps.Mattes.Miller....,1990-03-21T00:00:00Z
498,Grateful Dead Live at Copps Coliseum on 1990-0...,1990,"Hamilton, ON",gd1990-03-22.141931.FOB.Schoeps.Mattes.Miller....,1990-03-22T00:00:00Z


In [23]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{}:{}@deadhead-db.cplvgriavgfs.us-east-1.rds.amazonaws.com:3306/deadhead".format(db_username, db_password))

In [24]:
clean_concert_df.to_sql('concert',engine, if_exists='replace',index_label='concert_id')

500

In [25]:
def getSongs(concert_id, concert_identifier):
  url = "https://archive.org/details/{}".format(concert_identifier)
  songs = requests.get(url+'?output=json')
  songjson = songs.json()

  songs = []
  for key, value in songjson['files'].items():
      if value['format'] == 'Flac' or value['format'] == 'VBR MP3':
        value['archive_id'] = key
        value['url'] = "https://ia804601.us.archive.org/0/items/{}{}".format(concert_identifier, key)
        songs.append(value)
        print(key, value)
  try:
    del songs[0] # Get rid of the tuning song
  except Exception:
    print('Concert doesnt have a song', end='\r')

  songs_df = pd.DataFrame(songs)
  songs_df = songs_df[['title','format','archive_id', 'url']]
  songs_df['concert_id'] = concert_id
  songs_df['corrupted'] = 'No'
  return songs_df

In [10]:
for ind in clean_concert_df.index:
    print('ID:',ind, "Getting songs for", clean_concert_df['title'][ind], 'identifier:',clean_concert_df['identifier'][ind], end='\r')
    try:
        df = getSongs(ind, clean_concert_df['identifier'][ind])
    except Exception:
        continue

    df.to_sql('song',engine, if_exists='append',index=False)